In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
def cleaning():
    df = pd.read_csv("crop_yield.csv", header=0)
    df.drop(columns=["Crop_Year", "Production"], inplace=True)
    df = df[(df["Yield"] > 0.1) & (df["Yield"] <= 150)]
    df = df[df["Crop"] != "Coconut"]
    df["Season"] = df["Season"].str.replace(" ", "")
    return df

In [5]:
df = cleaning()
df.to_csv("cleaned_crop.csv", index_label='ID')

In [6]:
from sklearn.model_selection import train_test_split

# Load the data
df = pd.read_csv("cleaned_crop.csv", index_col="ID")

# # First, let's split the original dataframe into two dataframes, one containing 80% of the data and the other containing 20% of the data.
# df_80, df_20 = train_test_split(df, test_size=0.2, stratify=df["Crop"], random_state=0)

# # Now let's split df_80 into two dataframes, each containing 40% of the original data.
# df_40_1, df_40_2 = train_test_split(
#     df_80, test_size=0.5, stratify=df_80["Crop"], random_state=0
# )

# # Split df_40_1 into two dataframes, each containing 20% of the original data.
# train_1, valid_1 = train_test_split(
#     df_40_1, test_size=0.5, stratify=df_40_1["Crop"], random_state=0
# )

# # Split df_40_2 into two dataframes, each containing 20% of the original data.
# train_2, train_3 = train_test_split(
#     df_40_2, test_size=0.5, stratify=df_40_2["Crop"], random_state=0
# )

# # Now df_20, train_1, valid_1, train_2, and train_3 are your five dataframes,
# # and you can save them to CSV files as follows:
# test_1 = df_20
# train_1.to_csv("train_1.csv")
# train_2.to_csv("train_2.csv")
# train_3.to_csv("train_3.csv")
# valid_1.to_csv("valid_1.csv")
# test_1.to_csv("test_1.csv")

In [39]:
# file_path = ["train_1.csv", "train_2.csv", "train_3.csv"]


# def csv_reader_dataset(
#     filepaths,
#     n_readers=5,
#     n_read_threads=tf.data.AUTOTUNE,
#     shuffle_buffer_size=10_000,
#     seed=42,
#     batch_size=32,
# ):
#     dataset = tf.data.Dataset.list_files(filepaths)

#     dataset = dataset.interleave(
#         lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
#         cycle_length=n_readers, num_parallel_calls=n_read_threads)

#     dataset = dataset.shuffle(shuffle_buffer_size, seed = seed)

#     return dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


# test_df = csv_reader_dataset(file_path)

# print(test_df.take(2))

<_TakeDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>


In [7]:
import numpy as np


X = df.drop(columns="Yield")
y = df["Yield"].copy()

from sklearn.model_selection import train_test_split

# Split the original data into training (70%) and temporary test (30%) sets
X_train, X_temp, Y_train, Y_temp = train_test_split(
    X, y, test_size=0.3, stratify=df["Crop"], random_state=0
)

# Now split the temporary test set into validation (15%) and test (15%) sets
X_valid, X_test, Y_valid, Y_test = train_test_split(
    X_temp, Y_temp, test_size=0.5, stratify=X_temp["Crop"], random_state=0
)


def apply_log1p_to_numeric(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].apply(np.log1p)
    return df


X_train_logged = apply_log1p_to_numeric(X_train.copy())
X_valid_logged = apply_log1p_to_numeric(X_valid.copy())
X_test_logged = apply_log1p_to_numeric(X_test.copy())

In [98]:
import tensorflow_hub as hub


norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(
    X_train_logged.select_dtypes(include=[np.number])
)  # Correctly adapt on the logged training data

# Create one-hot encoding layers and adapt them
one_hot_layer_1 = tf.keras.layers.StringLookup(output_mode="one_hot")
one_hot_layer_1.adapt(df["State"])
one_hot_layer_2 = tf.keras.layers.StringLookup(output_mode="one_hot")
one_hot_layer_2.adapt(df["Season"])

embeddings_layer = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim50/2")

input_numeric = tf.keras.Input(
    shape=(X_train.shape[1] - 3,), dtype=tf.float32, name="NumericData"
)
input_text1 = tf.keras.Input(shape=(1,), dtype=tf.string, name="State")
input_text2 = tf.keras.Input(shape=(1,), dtype=tf.string, name="Season")
input_text3 = tf.keras.Input(shape=(1,), dtype=tf.string, name="Crop")


norm_numeric = norm_layer(input_numeric)
ohe_hot_state = one_hot_layer_1(input_text1)
one_hot_season = one_hot_layer_2(input_text2)

################
input_text3 = tf.keras.Input(shape=(1,), dtype=tf.string, name="Crop")
input_text3_squeezed = tf.squeeze(input_text3, axis=1)
embedded_crop = embeddings_layer(input_text3_squeezed)
# embedded_crop = embeddings_layer(input_text3)
################



concat_input = tf.keras.layers.concatenate(
    [norm_numeric, ohe_hot_state, one_hot_season, embedded_crop]
)

dense_1 = tf.keras.layers.Dense(128, activation="relu", kernel_initializer="he_normal")(
    concat_input
)
dense_2 = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(
    dense_1
)
output_layer = tf.keras.layers.Dense(1)(dense_2)

model = tf.keras.Model(
    inputs=[input_numeric, input_text1, input_text2, input_text3], outputs=output_layer
)

# Comile the model
# optimizer=tf.keras.optimizers.legacy.SGD(momentum=0.9, nesterov=True)
model.compile(
    optimizer='adam',
    loss="mse",
    metrics=["RootMeanSquaredError"],
)


# Fit the model

input_data = {
    "NumericData": X_train_logged.select_dtypes(include=[np.number]),
    "State": X_train_logged["State"],
    "Season": X_train_logged["Season"],
    "Crop": X_train_logged["Crop"],
}


validation_data = (
    {
        "NumericData": X_valid_logged.select_dtypes(include=[np.number]),
        "State": X_valid_logged["State"],
        "Season": X_valid_logged["Season"],
        "Crop": X_valid_logged ["Crop"],
    },
    Y_valid,
)

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)


def one_cycle_scheduler(epoch, lr):
    max_lr = 0.01  # Set your maximum learning rate
    base_lr = 0.001  # Set your base (minimum) learning rate
    total_epochs = 20  # Set your total number of training epochs

    if epoch < total_epochs / 2:
        # Linearly increase the learning rate from base_lr to max_lr
        return base_lr + (max_lr - base_lr) * (epoch / (total_epochs / 2))
    else:
        # Linearly decrease the learning rate from max_lr to base_lr
        return max_lr - (max_lr - base_lr) * (
            (epoch - total_epochs / 2) / (total_epochs / 2)
        )

# Create the LearningRateScheduler callback
lr_schedule_callback = tf.keras.callbacks.LearningRateScheduler(one_cycle_scheduler)

history = model.fit(
    input_data,
    Y_train,
    epochs=20,
    validation_data=validation_data,
    callbacks=[early_stopping, lr_schedule_callback],
)

Epoch 1/20
422/422 [==============================] - 8s 15ms/step - loss: 83.8844 - root_mean_squared_error: 9.1588 - val_loss: 55.2190 - val_root_mean_squared_error: 7.4309 - lr: 0.0010
Epoch 2/20
422/422 [==============================] - 3s 8ms/step - loss: 54.3094 - root_mean_squared_error: 7.3695 - val_loss: 49.5871 - val_root_mean_squared_error: 7.0418 - lr: 0.0019
Epoch 3/20
422/422 [==============================] - 3s 8ms/step - loss: 67.8975 - root_mean_squared_error: 8.2400 - val_loss: 54.9844 - val_root_mean_squared_error: 7.4151 - lr: 0.0028
Epoch 4/20
422/422 [==============================] - 3s 8ms/step - loss: 90.7543 - root_mean_squared_error: 9.5265 - val_loss: 107.8284 - val_root_mean_squared_error: 10.3840 - lr: 0.0037
Epoch 5/20
422/422 [==============================] - 3s 8ms/step - loss: 172.7436 - root_mean_squared_error: 13.1432 - val_loss: 86.0633 - val_root_mean_squared_error: 9.2770 - lr: 0.0046
Epoch 6/20
422/422 [==============================] - 3s 8ms

In [96]:
embeddings_layer(X_train_logged["Crop"])

<tf.Tensor: shape=(13497, 50), dtype=float32, numpy=
array([[ 0.2216289 , -0.33971202,  0.199434  , ...,  0.04292737,
        -0.2232489 , -0.19333361],
       [ 0.07214864, -0.04171149, -0.00954658, ...,  0.15353486,
        -0.00463091,  0.10250401],
       [-0.0877428 , -0.18014443,  0.07023089, ...,  0.14613886,
        -0.06293976,  0.09830101],
       ...,
       [ 0.2216289 , -0.33971202,  0.199434  , ...,  0.04292737,
        -0.2232489 , -0.19333361],
       [-0.02339478, -0.19061647,  0.03753536, ..., -0.02141087,
         0.06894715, -0.11102317],
       [-0.00661174, -0.21552294, -0.21807763, ...,  0.18313994,
        -0.09194405, -0.06275211]], dtype=float32)>